In [ ]:
from pathlib import Path

import numpy as np
import osmnx as ox
from common import load_crs, load_stops
from determine_distance_from_center import (
    calculate_medoid,
    determine_city_centroid_by_landuse,
    read_boundary,
)

ox.settings.use_cache = True
ox.settings.log_console = False

crs = load_crs()

In [ ]:
CITY = "helsinki"
VERSION = ""
# possible values: Eigenvector Centrality, Degree Centrality, Closeness Centrality, Betweenness Centrality
CENTRALITY = "Betweenness Centrality"

In [ ]:
stops = load_stops(CITY, VERSION)
stops.to_crs(crs[CITY], inplace=True)

In [ ]:
landuse_centroid = determine_city_centroid_by_landuse(read_boundary(CITY), crs[CITY])

In [7]:
maxc = stops[CENTRALITY].max()
medoid_id = calculate_medoid(stops[stops["Betweenness Centrality"] == maxc])
medoid = stops[stops["stop_id"] == medoid_id]["geometry"].tolist()[0]
centroid = stops[stops["Betweenness Centrality"] == maxc].union_all().centroid

In [8]:
stops["distance_from_largest_betweenness_medoid"] = stops["geometry"].apply(
    lambda x: np.round(x.distance(medoid) / 1000, 3)
)
stops["distance_from_largest_betweenness_centroid"] = stops["geometry"].apply(
    lambda x: np.round(x.distance(centroid) / 1000, 3)
)
stops["distance_from_landuse_centroid"] = stops["geometry"].apply(
    lambda x: np.round(x.distance(landuse_centroid) / 1000, 3)
)

In [ ]:
distance = stops[
    [
        "stop_id",
        "distance_from_largest_betweenness_centroid",
        "distance_from_largest_betweenness_medoid",
        "distance_from_landuse_centroid",
    ]
].copy()
path = f"../output/{CITY}/{VERSION}"
Path(path).mkdir(parents=True, exist_ok=True)
distance.dropna(subset=["stop_id"]).to_csv(f"{path}/distance.csv", index=False)